In [42]:
import cv2
import numpy as np

##
# @brief Open a video file for reading and set up a writer for output.
#
# @details
# - Input video: "raw_video_feed.mp4"
# - Output video: "output.mp4"
# - Frame size: 320 x 240
# - Codec: mp4v

# Open the input video
video = cv2.VideoCapture("raw_video_feed.mp4")

# Define frame dimensions
width = 320
height = 240

# Get input video FPS
fps = video.get(cv2.CAP_PROP_FPS)

# Define codec and create output writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4', fourcc, fps, (width, height))


In [38]:
##
# @brief Resize, convert to grayscale, and apply binary threshold.
# @param frame Input image frame.
# @return Thresholded binary image.

def thresh_image(frame):
    frame = cv2.resize(frame, (width, height))
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    _, threshed = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY)
    return threshed


In [39]:
##
# @brief Find midpoint of low-intensity region in a row of the frame.
# @param frame Input image frame.
# @return Column index of midpoint.

def find_dot_coord(frame):
    start = 0
    end = width - 1
    started = False
    finished = False

    threshed = thresh_image(frame)
    row = threshed[height - 20, :]

    for i, pixel in enumerate(row):
        if not started and pixel < 255:
            start = i
            started = True

        if started and not finished and pixel > 0:
            end = i
            finished = True

    return (end + start) // 2


In [40]:
##
# @brief Draw a red circle at the detected dot coordinate.
# @param frame Input image frame.
# @return Frame with circle drawn.

def place_circle(frame):
    row_index = height - 20
    col_index = find_dot_coord(frame)

    cv2.circle(frame, (col_index, row_index), 20, (0, 0, 255), -1)
    
    return frame


In [41]:
##
# @brief Read frames, process with place_circle, and write to output.

while video.isOpened():

    ret, frame = video.read()
    if not ret:
        break

    frame = place_circle(frame)
    
    out.write(frame)

video.release()
out.release()
